In [17]:
# Cell 1: Enhanced Setup and Imports
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import enhanced modules
from src.data.dataset import ChestXrayDataset, create_data_splits, calculate_class_weights
from src.data.dataloader import create_rtx3060_dataloaders
from src.models.densenet121_enhanced import create_enhanced_model_for_rtx3060
from src.models.trainer_enhanced import EnhancedRTX3060Trainer

print("✅ Full Enhanced + Speed Optimized setup")

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

✅ Full Enhanced + Speed Optimized setup
🚀 Using device: cuda
   GPU: NVIDIA GeForce RTX 3060
   VRAM: 12.9 GB


In [18]:
# Cell 2: Load Data Pipeline
BASE_PATH = Path(r"D:/Projects/CLARITY/Model/Dataset/archive")  # Update this!

print("🔄 Loading data pipeline...")

# Load metadata
data_entry_path = BASE_PATH / "Data_Entry_2017.csv"
df = pd.read_csv(data_entry_path)
print(f"✅ Metadata loaded: {len(df):,} entries")

# Recreate image mapping
image_mapping = {}
for main_folder in sorted(BASE_PATH.iterdir()):
    if main_folder.is_dir() and main_folder.name.startswith('images_'):
        images_subfolder = main_folder / 'images'
        if images_subfolder.exists():
            for img_file in images_subfolder.glob("*.png"):
                image_name = img_file.name
                if image_name not in image_mapping:
                    image_mapping[image_name] = img_file

print(f"✅ Image mapping: {len(image_mapping):,} images")

# Create data splits (consistent with previous experiments)
train_df, val_df, test_df = create_data_splits(df, 
                                               test_size=0.15,
                                               val_size=0.10,
                                               random_seed=42)

print(f"✅ Data splits: Train({len(train_df):,}) Val({len(val_df):,}) Test({len(test_df):,})")

🔄 Loading data pipeline...
✅ Metadata loaded: 112,120 entries
✅ Image mapping: 112,120 images
Patient-level data splits:
  Train: 83,847 images from 23,105 patients (74.8%)
  Val:   11,550 images from 3,080 patients (10.3%)
  Test:  16,723 images from 4,620 patients (14.9%)
✅ No patient overlap verified - clean splits!
✅ Data splits: Train(83,847) Val(11,550) Test(16,723)


In [19]:
# Cell 3: Full Enhanced + Speed Optimized Settings
print("🔄 Creating full enhanced with speed optimizations...")

# OPTIMIZED SETTINGS FOR FULL ENHANCED
OPTIMAL_IMAGE_SIZE = 448    # Sweet spot: better than 384, faster than 512
OPTIMAL_BATCH_SIZE = 8      # Optimal for 448px on RTX 3060
OPTIMAL_WORKERS = 4         # Balanced CPU usage
OPTIMAL_ACCUMULATION = 3    # Effective batch = 24

# Create datasets with optimized settings
train_dataset = ChestXrayDataset(
    df=train_df,
    image_mapping=image_mapping,
    image_size=OPTIMAL_IMAGE_SIZE,
    is_training=True,
    augmentation_prob=0.85  # Strong augmentation for better generalization
)

val_dataset = ChestXrayDataset(
    df=val_df,
    image_mapping=image_mapping,
    image_size=OPTIMAL_IMAGE_SIZE,
    is_training=False
)

test_dataset = ChestXrayDataset(
    df=test_df,
    image_mapping=image_mapping,
    image_size=OPTIMAL_IMAGE_SIZE,
    is_training=False
)

print(f"✅ Full enhanced datasets created!")
print(f"   Resolution: {OPTIMAL_IMAGE_SIZE}×{OPTIMAL_IMAGE_SIZE}")
print(f"   Augmentation: 85%")
print(f"   Expected: ~12-15 min/epoch (vs 30 min at 512px)")

🔄 Creating full enhanced with speed optimizations...
Dataset created with 83847 samples
Training mode: True
Image size: 448x448

Label matrix created: (83847, 15)
Positive samples per class:
  No Finding...............  45146
  Atelectasis..............   8720
  Cardiomegaly.............   2019
  Effusion.................  10071
  Infiltration.............  14772
  Mass.....................   4477
  Nodule...................   4691
  Pneumonia................   1062
  Pneumothorax.............   3981
  Consolidation............   3458
  Edema....................   1738
  Emphysema................   1794
  Fibrosis.................   1236
  Pleural_Thickening.......   2562
  Hernia...................    171
Transforms created for training
Dataset created with 11550 samples
Training mode: False
Image size: 448x448

Label matrix created: (11550, 15)
Positive samples per class:
  No Finding...............   6197
  Atelectasis..............   1148
  Cardiomegaly.............    331
  Effusi

In [20]:
# Cell 4: Speed-Optimized DataLoaders for Full Enhanced
print("🔄 Creating optimized dataloaders...")

# Enhanced class weights
class_weights = calculate_class_weights(train_dataset.labels, method='inverse_freq')

# Create optimized dataloaders
train_loader, val_loader, test_loader = create_rtx3060_dataloaders(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    batch_size=OPTIMAL_BATCH_SIZE,
    num_workers=OPTIMAL_WORKERS,
    use_weighted_sampling=True
)

# Memory calculation
memory_per_batch_gb = (OPTIMAL_BATCH_SIZE * 3 * OPTIMAL_IMAGE_SIZE * OPTIMAL_IMAGE_SIZE * 4) / 1e9
print(f"\n📊 Optimized Memory Usage:")
print(f"   Per batch: ~{memory_per_batch_gb:.2f} GB")
print(f"   Effective batch: {OPTIMAL_BATCH_SIZE * OPTIMAL_ACCUMULATION}")
print(f"   Expected time: 12-15 min/epoch")
print(f"   Memory safe: ✅ ({memory_per_batch_gb * 3:.1f}GB < 12GB)")

🔄 Creating optimized dataloaders...

Class weights (inverse_freq):
--------------------------------------------------
No Finding...............    0.031 (pos:  45146)
Atelectasis..............    0.158 (pos:   8720)
Cardiomegaly.............    0.684 (pos:   2019)
Effusion.................    0.137 (pos:  10071)
Infiltration.............    0.093 (pos:  14772)
Mass.....................    0.308 (pos:   4477)
Nodule...................    0.294 (pos:   4691)
Pneumonia................    1.299 (pos:   1062)
Pneumothorax.............    0.347 (pos:   3981)
Consolidation............    0.399 (pos:   3458)
Edema....................    0.794 (pos:   1738)
Emphysema................    0.769 (pos:   1794)
Fibrosis.................    1.116 (pos:   1236)
Pleural_Thickening.......    0.539 (pos:   2562)
Hernia...................    8.030 (pos:    171)
Creating RTX 3060 optimized dataloaders:
  Batch size: 8
  Num workers: 4
  Weighted sampling: True
  Weighted sampler created with 83847 samples
D

In [22]:
# Cell 5: Full Enhanced Model (FIXED - No JIT optimization)
print("🔄 Creating full enhanced model...")

# Create full enhanced model (keeping all advanced features)
model, criterion = create_enhanced_model_for_rtx3060(
    num_classes=15,
    class_weights=class_weights,
    model_size="standard"  # Standard for speed, still full enhanced
)

# Move to device
model = model.to(device)
criterion = criterion.to(device)

# Enable CuDNN optimization (safe and effective)
torch.backends.cudnn.benchmark = True  # This is fine
torch.backends.cudnn.deterministic = False  # Allow faster algorithms

# Print model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Full Enhanced Model (Speed Optimized):")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model size: ~{total_params * 4 / 1e6:.1f} MB")
print(f"   Attention modules: ✅ Enabled")
print(f"   Advanced loss: ✅ Triple-component")
print(f"   CuDNN optimized: ✅ Enabled")
print(f"   Ready for training: ✅")

🔄 Creating full enhanced model...
✅ Enhanced DenseNet121 Multi-Label Model created:
   Classes: 15
   Pretrained: True
   Dropout: 0.4
   Attention: True
   Feature Fusion: True
✅ Enhanced Focal Loss: alpha=0.25, gamma=2.5
✅ Asymmetric Loss: gamma_neg=4, gamma_pos=1, clip=0.05
✅ Enhanced Combined Loss:
   Focal: 0.400
   BCE: 0.400
   Asymmetric: 0.200
🚀 Enhanced RTX 3060 Model Configuration Complete!
   Model Size: standard
   Enhanced Features: Attention + Multi-layer Classifier
   Loss: Triple-component (Focal + BCE + Asymmetric)

📊 Full Enhanced Model (Speed Optimized):
   Total parameters: 8,798,193
   Trainable parameters: 8,798,193
   Model size: ~35.2 MB
   Attention modules: ✅ Enabled
   Advanced loss: ✅ Triple-component
   CuDNN optimized: ✅ Enabled
   Ready for training: ✅


In [23]:
# Cell 6: Progressive Training Setup (20→30→50 epochs)
print("🔄 Setting up progressive training...")

def create_progressive_trainer(stage="stage1", previous_trainer=None):
    """Create trainer for different stages of progressive training"""
    
    if stage == "stage1":
        # Stage 1: 20 epochs - Focus on rapid convergence
        config = {
            'learning_rate': 3e-4,           # Higher LR for initial learning
            'scheduler_type': 'onecycle',     # Aggressive scheduling
            'warmup_epochs': 2,              # Quick warmup
            'max_epochs': 20,                # First checkpoint
            'patience': 8,                   # Allow exploration
            'label_smoothing': 0.05,         # Light smoothing
            'accumulation_steps': OPTIMAL_ACCUMULATION,
            'mixed_precision': True
        }
        print(f"🎯 Stage 1 Config: 20 epochs, aggressive learning")
        
    elif stage == "stage2":
        # Stage 2: +10 epochs (30 total) - Refinement
        config = {
            'learning_rate': 1e-4,           # Reduced LR for refinement
            'scheduler_type': 'cosine',      # Smooth scheduling
            'warmup_epochs': 1,              # Minimal warmup
            'max_epochs': 10,                # Additional epochs
            'patience': 6,                   # Tighter patience
            'label_smoothing': 0.1,          # More smoothing
            'accumulation_steps': OPTIMAL_ACCUMULATION,
            'mixed_precision': True
        }
        print(f"🎯 Stage 2 Config: +10 epochs (30 total), refinement")
        
    else:  # stage3
        # Stage 3: +20 epochs (50 total) - Fine-tuning
        config = {
            'learning_rate': 5e-5,           # Low LR for fine-tuning
            'scheduler_type': 'plateau',     # Conservative scheduling
            'warmup_epochs': 0,              # No warmup needed
            'max_epochs': 20,                # Final epochs
            'patience': 10,                  # High patience for final gains
            'label_smoothing': 0.15,         # Heavy smoothing
            'accumulation_steps': OPTIMAL_ACCUMULATION,
            'mixed_precision': True
        }
        print(f"🎯 Stage 3 Config: +20 epochs (50 total), fine-tuning")
    
    trainer = EnhancedRTX3060Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        device=device,
        checkpoint_dir=f'../models/checkpoints_{stage}',
        **config
    )
    
    # Load previous state if continuing
    if previous_trainer and stage != "stage1":
        # Copy training history
        trainer.train_losses = previous_trainer.train_losses.copy()
        trainer.val_losses = previous_trainer.val_losses.copy()
        trainer.val_aucs = previous_trainer.val_aucs.copy()
        trainer.val_aps = previous_trainer.val_aps.copy()
        trainer.learning_rates = previous_trainer.learning_rates.copy()
        trainer.best_val_auc = previous_trainer.best_val_auc
        trainer.best_epoch = previous_trainer.best_epoch
    
    return trainer

print("✅ Progressive training setup complete!")
print("   Stage 1: 20 epochs - Fast convergence")
print("   Stage 2: 30 epochs - Refinement (if AUC > 0.72)")
print("   Stage 3: 50 epochs - Fine-tuning (if AUC > 0.76)")

🔄 Setting up progressive training...
✅ Progressive training setup complete!
   Stage 1: 20 epochs - Fast convergence
   Stage 2: 30 epochs - Refinement (if AUC > 0.72)
   Stage 3: 50 epochs - Fine-tuning (if AUC > 0.76)


In [24]:
# Cell 7: Stage 1 Training - 20 Epochs
print("🚀 Starting Stage 1: Full Enhanced Training (20 epochs)")
print("=" * 80)

import time
stage1_start = time.time()

# Create Stage 1 trainer
trainer_stage1 = create_progressive_trainer("stage1")

# Train Stage 1
print("🎯 Target for Stage 1: AUC > 0.72 (improvement > +0.03)")
stage1_auc = trainer_stage1.train(num_epochs=20, save_every=5)

stage1_time = time.time() - stage1_start

print(f"\n🏁 Stage 1 Results:")
print(f"=" * 50)
print(f"   AUC: {stage1_auc:.4f}")
print(f"   Baseline: 0.688")
print(f"   Improvement: +{stage1_auc - 0.688:.4f}")
print(f"   Training time: {stage1_time/3600:.2f} hours")
print(f"   Time per epoch: {stage1_time/60/20:.1f} minutes")

# Decision for Stage 2
if stage1_auc >= 0.72:
    print(f"✅ Stage 1 SUCCESS! AUC ≥ 0.72")
    print(f"   → Proceed to Stage 2 (30 epochs total)")
    proceed_stage2 = True
elif stage1_auc >= 0.70:
    print(f"🔶 Stage 1 PARTIAL SUCCESS! 0.70 ≤ AUC < 0.72")
    print(f"   → Consider Stage 2 for potential gains")
    proceed_stage2 = True
else:
    print(f"🔸 Stage 1 needs improvement. AUC < 0.70")
    print(f"   → May need hyperparameter adjustment")
    proceed_stage2 = False

# Save Stage 1 model regardless
stage1_model_name = f"clarity_stage1_auc{stage1_auc:.3f}_ep20.pth"
torch.save({
    'model_state_dict': model.state_dict(),
    'stage1_auc': stage1_auc,
    'stage': 'stage1_complete',
    'epochs': 20,
    'training_time_hours': stage1_time / 3600,
    'next_stage_recommended': proceed_stage2
}, f"../models/saved_models/{stage1_model_name}")

print(f"💾 Stage 1 model saved: {stage1_model_name}")

🚀 Starting Stage 1: Full Enhanced Training (20 epochs)
🎯 Stage 1 Config: 20 epochs, aggressive learning
✅ Mixed precision training enabled (FP16)
🚀 Enhanced RTX 3060 Trainer initialized:
   Device: cuda
   Batch size: 8
   Effective batch size: 24
   Learning rate: 0.0003
   Scheduler: onecycle
   Warmup epochs: 2
   Mixed precision: True
   Label smoothing: 0.05
🎯 Target for Stage 1: AUC > 0.72 (improvement > +0.03)
🚀 Starting enhanced training for 20 epochs
   Progressive training: False


💾 New best model saved: AUC = 0.6759
Epoch   1/20 | Train Loss: 0.1978 | Val Loss: 0.0803 | Val AUC: 0.6759 | Val AP: 0.1423 | LR: 9.39e-05 | Time: 1338.7s


💾 New best model saved: AUC = 0.7088
Epoch   2/20 | Train Loss: 0.1505 | Val Loss: 0.0768 | Val AUC: 0.7088 | Val AP: 0.2014 | LR: 2.52e-04 | Time: 1320.9s


💾 New best model saved: AUC = 0.7189
Epoch   3/20 | Train Loss: 0.1466 | Val Loss: 0.1026 | Val AUC: 0.7189 | Val AP: 0.1860 | LR: 4.68e-04 | Time: 1302.0s


💾 New best model saved: AUC = 0.7240
Epoch   4/20 | Train Loss: 0.1439 | Val Loss: 0.0740 | Val AUC: 0.7240 | Val AP: 0.1988 | LR: 6.84e-04 | Time: 1298.8s


...

Epoch  16/20 | Train Loss: 0.1178 | Val Loss: 0.0609 | Val AUC: 0.7648 | Val AP: 0.2377 | LR: 1.70e-04 | Time: 1233.9s


Epoch  17/20 | Train Loss: 0.1157 | Val Loss: 0.0615 | Val AUC: 0.7608 | Val AP: 0.2360 | LR: 9.83e-05 | Time: 1234.4s


Epoch  18/20 | Train Loss: 0.1141 | Val Loss: 0.0601 | Val AUC: 0.7647 | Val AP: 0.2369 | LR: 4.46e-05 | Time: 1234.1s


Epoch  19/20 | Train Loss: 0.1137 | Val Loss: 0.0604 | Val AUC: 0.7615 | Val AP: 0.2370 | LR: 1.13e-05 | Time: 1234.4s


Epoch  20/20 | Train Loss: 0.1132 | Val Loss: 0.0592 | Val AUC: 0.7579 | Val AP: 0.2358 | LR: 3.62e-09 | Time: 1246.4s

Per-class AUC scores (Epoch 20):
  🔶 No Finding............... 0.750
  🔶 Atelectasis.............. 0.741
  ✅ Cardiomegaly............. 0.902
  ✅ Effusion................. 0.855
  🔸 Infiltration............. 0.676
  🔶 Mass..................... 0.725
  🔸 Nodule................... 0.587
  🔸 Pneumonia................ 0.648
  ✅ Pneumothorax............. 0.841
  🔶 Consolidation............ 0.766
  ✅ Edema.................... 0.870
  ✅ Emphysema................ 0.925
  🔶 Fibrosis................. 0.729
  🔶 Pleural_Thickening....... 0.706
  🔸 Hernia................... 0.647


🎉 Enhanced training completed!
Total time: 7.05 hours
Best validation AUC: 0.7675 (Epoch 14)
Final learning rate: 3.62e-09
Training stages: 1

🏁 Stage 1 Results:
   AUC: 0.7675
   Baseline: 0.688
   Improvement: +0.0795
   Training time: 7.05 hours
   Time per epoch: 21.2 minutes
✅ Stage 1 SUCCESS! AUC ≥

In [36]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path

# Create save directory
save_dir = Path("../models/saved_models/enhanced_densenet121")
save_dir.mkdir(parents=True, exist_ok=True)

# Save model weights and architecture info
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': trainer_stage1.optimizer.state_dict(),
    'scheduler_state_dict': trainer_stage1.scheduler.state_dict(),
    'architecture': 'DenseNet121Enhanced',
    'num_classes': 15,
    'image_size': getattr(model, 'image_size', 448),
    'use_attention': getattr(model, 'use_attention', True),
    'feature_fusion': getattr(model, 'feature_fusion', True),
    'dropout_rate': 0.4,
    'pretrained': True
}, save_dir / "model.pth")

# Fix array length mismatch
train_losses = trainer_stage1.train_losses
val_losses = trainer_stage1.val_losses
val_aucs = trainer_stage1.val_aucs
val_aps = trainer_stage1.val_aps
learning_rates = trainer_stage1.learning_rates

# Find minimum length to ensure all arrays are same size
min_length = min(len(train_losses), len(val_losses), len(val_aucs), len(val_aps), len(learning_rates))

print(f"Array lengths: train={len(train_losses)}, val={len(val_losses)}, auc={len(val_aucs)}, ap={len(val_aps)}, lr={len(learning_rates)}")
print(f"Using minimum length: {min_length}")

# Save training history (CSV) with matched lengths
pd.DataFrame({
    'epoch': range(1, min_length + 1),
    'train_loss': train_losses[:min_length],
    'val_loss': val_losses[:min_length],
    'val_auc': val_aucs[:min_length],
    'val_ap': val_aps[:min_length],
    'learning_rate': learning_rates[:min_length]
}).to_csv(save_dir / "training_history.csv", index=False)

# Save key training metrics as numpy file
np.savez_compressed(
    save_dir / "training_metrics.npz",
    train_losses=np.array(train_losses),
    val_losses=np.array(val_losses),
    val_aucs=np.array(val_aucs),
    val_aps=np.array(val_aps),
    learning_rates=np.array(learning_rates)
)

print("✅ Model, training history, and related data saved to:", save_dir)
print(f"   Epochs saved: {min_length}")
print(f"   Final AUC: {val_aucs[-1]:.4f}" if val_aucs else "   No AUC data")

Array lengths: train=20, val=20, auc=20, ap=20, lr=20
Using minimum length: 20
✅ Model, training history, and related data saved to: ..\models\saved_models\enhanced_densenet121
   Epochs saved: 20
   Final AUC: 0.7579
